In [ ]:
#!pip install transformers requests beautifulsoup4 pandas numpy


This will give a sentiment score between 1 and 5

In [60]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

In [61]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model =  AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [62]:
tokens = tokenizer.encode('I had a great time at the new resort', return_tensors='pt')

In [63]:
tokens

tensor([[  101,   151, 10407,   143, 11838, 10573, 10160, 10103, 10246, 33016,
           102]])

In [64]:
result = model(tokens)

In [65]:
result.logits

tensor([[-1.7287, -1.3514, -0.1626,  1.1591,  1.5593]],
       grad_fn=<AddmmBackward0>)

In [66]:
int(torch.argmax(result.logits))+1

5

In [67]:
tokens = tokenizer.encode('', return_tensors='pt')

In [68]:
result = model(tokens)

In [69]:
result.logits

tensor([[-0.5486, -0.4852,  0.2038,  0.3929,  0.2652]],
       grad_fn=<AddmmBackward0>)

In [70]:
int(torch.argmax(result.logits))+1

4

In [71]:
tokens = tokenizer.encode('Meh, it was ok', return_tensors='pt')

In [72]:
result = model(tokens)

In [73]:
result.logits

tensor([[-0.7896,  1.6718,  2.7634, -0.2691, -2.8158]],
       grad_fn=<AddmmBackward0>)

In [74]:
int(torch.argmax(result.logits))+1

3

In [75]:
tokens = tokenizer.encode('Meh, it was ok', return_tensors='pt')
result = model(tokens)
result.logits
int(torch.argmax(result.logits))+1



3

# Look at Some Reviews

In [76]:
#r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
r = requests.get('https://www.yelp.com/biz/wegmans-bayou-louisiana-kitchen-roswell')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [78]:
results[0].text

'They have a great menu and the food is delicious!Our waiter was so friendly, \xa0accommodating and knowledgeable. \xa0Definitely will be back!!'

# Load Reviews into DataFrame and Score

In [79]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [80]:
df.head()

,review
0,They have a great menu and the food is delicio...
1,Looking for a po' boy sandwich? You've come to...
2,I hate leaving 1-star reviews. You could easil...
3,Food is always delicious. Never disappoints. O...
4,Great location with various sitting options (b...


In [81]:
df.tail()

,review
5,Nice place in Roswell. The chargrilled oysters...
6,Awesome gem in downtown Roswell. Wonderful amb...
7,Enjoyed visiting this restaurant after six yea...
8,We came here for my birthday on 7/15 for the f...
9,The best way I can describe the food is OMG......


In [82]:
def sentiment_score(review):
    tokens = tokenizer.encode(review,return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [83]:
score = sentiment_score(df['review'].iloc[9])

In [84]:
score

5

In [85]:
df['sentiment']= df['review'].apply(lambda x: sentiment_score(x[:512]))

In [86]:
df

,review,sentiment
0,They have a great menu and the food is delicio...,5
1,Looking for a po' boy sandwich? You've come to...,4
2,I hate leaving 1-star reviews. You could easil...,1
3,Food is always delicious. Never disappoints. O...,5
4,Great location with various sitting options (b...,5
5,Nice place in Roswell. The chargrilled oysters...,5
6,Awesome gem in downtown Roswell. Wonderful amb...,5
7,Enjoyed visiting this restaurant after six yea...,4
8,We came here for my birthday on 7/15 for the f...,4
9,The best way I can describe the food is OMG......,5


In [87]:
df['review'].iloc[9]

"The best way I can describe the food is OMG...it's AMAZING...also we had the world's best server ... ask for Eric he is the man!!!"